In [1]:
%reload_ext autoreload
%autoreload 2

import os
import json
from pathlib import Path
import multiprocessing

import jax
import jax.numpy as jnp
import pandas as pd
import numpy as np
import numpyro
import numpyro.distributions as dist

from hbmep.config import Config
from hbmep.model import Baseline
from hbmep.model.utils import Site as site
from hbmep.utils.constants import RECTIFIED_LOGISTIC

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()


In [2]:
# src = "/home/vishu/data/hbmep-processed/J_RCML_000/ground"

# df = pd.read_csv(os.path.join(src, "data.csv"))
# mat = np.load(os.path.join(src, "mat.npy"))
# auc_window = np.load(os.path.join(src, "auc_window.npy"))

# f = open(os.path.join(src, "muscles_map.json"))
# muscles_map = json.load(f)

# muscles_map

In [3]:
# """ Filter """
# sub = [1, 2, 5]
# mat = mat[..., np.array(sub) - 1]

# muscles_map = {u: v for u, v in muscles_map.items() if int(u.split("_")[-1]) in sub}

# """ Save """
# dst = "/home/vishu/data/hbmep-processed/J_RCML_000/ground/1_2_5"

# df.to_csv(os.path.join(dst, "data.csv"), index=False)

# np.save(os.path.join(dst, "mat.npy"), mat)
# np.save(os.path.join(dst, "auc_window.npy"), np.array(auc_window))

# f = open(os.path.join(dst, "muscles_map.json"), "w")
# f.write(json.dumps(muscles_map))
# f.close;

In [4]:
root_path = Path(os.getcwd()).parent.parent.parent.absolute()
toml_path = os.path.join(root_path, "configs/ground_ADM_Biceps_ECR.toml")

config = Config(toml_path=toml_path)
config.BUILD_DIR = "/home/vishu/repos/hbmep-paper/reports/J_RCML_000/ground/1_2_5/nb"


2023-07-25 11:07:33,658 - hbmep.config - INFO - Verifying configuration ...
2023-07-25 11:07:33,659 - hbmep.config - INFO - Success!


In [5]:
class RectifiedLogistic(Baseline):
    LINK = RECTIFIED_LOGISTIC

    def __init__(self, config: Config):
        super(RectifiedLogistic, self).__init__(config=config)

    def _model(self, subject, features, intensity, response_obs=None):
        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)

        n_data = intensity.shape[0]
        n_subject = np.unique(subject).shape[0]
        n_feature0 = np.unique(feature0).shape[0]

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate(site.n_subject, n_subject, dim=-2):
                """ Hyper-priors """
                mu_a = numpyro.sample(
                    site.mu_a,
                    dist.TruncatedNormal(150, 50, low=0)
                )
                sigma_a = numpyro.sample(site.sigma_a, dist.HalfNormal(50))

                sigma_b = numpyro.sample(site.sigma_b, dist.HalfNormal(0.1))

                sigma_L = numpyro.sample(site.sigma_L, dist.HalfNormal(0.05))
                sigma_H = numpyro.sample(site.sigma_H, dist.HalfNormal(5))
                sigma_v = numpyro.sample(site.sigma_v, dist.HalfNormal(10))

                with numpyro.plate("n_feature0", n_feature0, dim=-3):
                    """ Priors """
                    a = numpyro.sample(
                        site.a,
                        dist.TruncatedNormal(mu_a, sigma_a, low=0)
                    )
                    b = numpyro.sample(site.b, dist.HalfNormal(sigma_b))

                    L = numpyro.sample(site.L, dist.HalfNormal(sigma_L))
                    H = numpyro.sample(site.H, dist.HalfNormal(sigma_H))
                    v = numpyro.sample(site.v, dist.HalfNormal(sigma_v))

                    g_1 = numpyro.sample("g_1", dist.Exponential(1 / 100))
                    g_2 = numpyro.sample("g_2", dist.Exponential(1 / 100))

        """ Model """
        mu = numpyro.deterministic(
            site.mu,
            L[feature0, subject]
            + jnp.maximum(
                0,
                -1
                + (H[feature0, subject] + 1)
                / jnp.power(
                    1
                    + (jnp.power(1 + H[feature0, subject], v[feature0, subject]) - 1)
                    * jnp.exp(-b[feature0, subject] * (intensity - a[feature0, subject])),
                    1 / v[feature0, subject]
                )
            )
        )
        beta = numpyro.deterministic(
            site.beta,
            g_1[feature0, subject] + g_2[feature0, subject] * (1 / mu)
        )

        with numpyro.plate(site.data, n_data):
            return numpyro.sample(
                site.obs,
                dist.Gamma(concentration=mu * beta, rate=beta).to_event(1),
                obs=response_obs
            )


model = RectifiedLogistic(config=config)

In [6]:
# df = pd.read_csv(model.csv_path)

# sub = ["amap01", "amap02", "amap03", "amap04"]
# # sub += ["amap05", "amap06", "amap07", "amap08"]

# # sub = ["amap01"]

# ind = df.participant.isin(sub)

# df = df[ind].copy()
# df.reset_index(drop=True, inplace=True)
# print(df.shape)

# df, encoder_dict = model.load(df=df)


In [7]:
df, encoder_dict = model.load()

2023-07-25 11:07:34,402 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/J_RCML_000/ground/1_2_5/nb
2023-07-25 11:07:34,403 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/ground/1_2_5/nb
2023-07-25 11:07:34,403 - hbmep.dataset.core - INFO - Reading data from /home/vishu/data/hbmep-processed/J_RCML_000/ground/1_2_5/data.csv ...
2023-07-25 11:07:34,415 - hbmep.dataset.core - INFO - Processing data ...
2023-07-25 11:07:34,418 - hbmep.utils.utils - INFO - func:load took: 0.02 sec


In [8]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-07-25 11:07:35,696 - hbmep.model.baseline - INFO - Running inference with rectified_logistic ...


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
mcmc.print_summary(prob=.95)



                mean       std    median      2.5%     97.5%     n_eff     r_hat
  H[0,0,0]      0.74      0.03      0.74      0.69      0.80  13713.67      1.00
  H[0,0,1]      2.30      1.41      1.90      0.81      4.93   5818.08      1.00
  H[0,0,2]      0.58      0.04      0.57      0.50      0.65  10518.67      1.00
  H[0,1,0]      3.32      3.75      2.03      0.04     10.95   7311.43      1.00
  H[0,1,1]      3.48      2.05      2.81      1.43      7.48   4551.23      1.00
  H[0,1,2]      1.85      2.28      1.05      0.21      6.08   4502.11      1.00
  H[0,2,0]      2.74      0.31      2.70      2.18      3.37   7291.64      1.00
  H[0,2,1]      2.37      0.53      2.25      1.57      3.36   5111.76      1.00
  H[0,2,2]      1.34      0.06      1.33      1.22      1.45  13635.48      1.00
  H[0,3,0]      3.83      1.96      3.24      1.54      7.65   6258.21      1.00
  H[0,3,1]      5.83      0.34      5.81      5.19      6.50  11998.86      1.00
  H[0,3,2]     10.19      0

In [ ]:
model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-25 10:49:43,331 - hbmep.model.baseline - INFO - Rendering recruitment curves ...


2023-07-25 10:52:45,309 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/ground/1_2_5/nb/recruitment_curves.pdf
2023-07-25 10:52:45,309 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 3 min and 1.98 sec


In [ ]:
model.render_predictive_check(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-25 10:53:43,256 - hbmep.model.baseline - INFO - Rendering Posterior Predictive Check ...


2023-07-25 10:57:24,891 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/ground/1_2_5/nb/posterior_predictive_check.pdf
2023-07-25 10:57:24,896 - hbmep.utils.utils - INFO - func:render_predictive_check took: 3 min and 41.64 sec


In [ ]:
model.save(mcmc=mcmc)

2023-07-25 10:57:43,357 - hbmep.model.baseline - INFO - Saving inference data ...
2023-07-25 10:58:43,007 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/ground/1_2_5/nb/mcmc.nc
2023-07-25 10:58:43,008 - hbmep.model.baseline - INFO - Rendering convergence diagnostics ...


2023-07-25 11:00:25,849 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/ground/1_2_5/nb/diagnostics.csv
2023-07-25 11:00:25,849 - hbmep.model.baseline - INFO - Evaluating model ...
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:1037: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:803: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.7 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
2023-07-25 11:00:38,231 - hbmep.model.baseline - INFO - ELPD LOO (Log): 5945.10
/home/vishu/r